<a href="https://colab.research.google.com/github/knsg16/scraping-hand/blob/main/%E3%83%8F%E3%83%B3%E3%83%89%E3%81%AE%E6%A4%9C%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 特定のspanタグを持つ要素を検索
    span_elements = soup.find_all('span', class_='post-list-cat category-jrp')

    for span in span_elements:
        if span.get_text().strip() == 'Jリーグジャッジリプレイ':
            related_url = span.find_parent('a')['href']

            # 関連するページを取得して解析
            related_response = requests.get(related_url)
            related_soup = BeautifulSoup(related_response.text, 'html.parser')

            # class="concept-box1"を持つ要素の中に「ハンド」が含まれているか確認
            for concept_box in related_soup.find_all('div', class_='concept-box1'):
                if 'ハンド' in concept_box.get_text():
                    print(f'The word "ハンド" found in {related_url}')
                    break  # 「ハンド」を見つけたらループを終了

def get_pagination_urls(base_url):
    """ページネーションのURLを取得する関数"""
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    pagination = soup.find('ul', class_='pagination ef')
    urls = []

    if pagination:
        links = pagination.find_all('a', class_='inactive')
        urls = [link.get('href') for link in links]

    return urls

# 基本URL
base_url = ''

# ページネーションURLのリストを取得
pagination_urls = get_pagination_urls(base_url)

# 最初のページをスクレイピング
scrape_page(base_url)

# 残りのページをスクレイピング
for url in pagination_urls:
    print(f'Scraping {url}')
    scrape_page(url)
    time.sleep(1)  # サーバーに負荷をかけないための遅延


The word "ハンド" found in https://tarutablog.com/jrp-2023-25/
The word "ハンド" found in https://tarutablog.com/jrp-2023-23/
Scraping https://tarutablog.com/category/reysol/zakki/jrp/page/2/
The word "ハンド" found in https://tarutablog.com/jrp-2023-21/
The word "ハンド" found in https://tarutablog.com/jrp-2023-18/
The word "ハンド" found in https://tarutablog.com/jrp-2023-16/
The word "ハンド" found in https://tarutablog.com/jrp-2023-14/
The word "ハンド" found in https://tarutablog.com/jrp-2023-13/
The word "ハンド" found in https://tarutablog.com/jrp-2023-12/
Scraping https://tarutablog.com/category/reysol/zakki/jrp/page/3/
The word "ハンド" found in https://tarutablog.com/jrp-2023-09/
The word "ハンド" found in https://tarutablog.com/jrp-2023-08/
The word "ハンド" found in https://tarutablog.com/jrp-2023-06/
The word "ハンド" found in https://tarutablog.com/jrp-2023-02/
Scraping https://tarutablog.com/category/reysol/zakki/jrp/page/4/
The word "ハンド" found in https://tarutablog.com/jrp-2022-29/
The word "ハンド" found i